# Using orai-nlp/Llama-eus-8B

**references:**

* https://huggingface.co/orai-nlp/Llama-eus-8B
* https://huggingface.co/docs/transformers/llm_tutorial

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import json
import numpy as np
import gc

Mounted at /content/drive


In [ ]:
#!pip install transformers bitsandbytes>=0.39.0

In [ ]:
from transformers import AutoModelForCausalLM

In [ ]:
#model = AutoModelForCausalLM.from_pretrained("orai-nlp/Llama-eus-8B", device_map="auto", load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained("orai-nlp/Llama-eus-8B", device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/872 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer
from accelerate.test_utils.testing import get_backend

DEVICE, _, _ = get_backend() # automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)
print(DEVICE)
tokenizer = AutoTokenizer.from_pretrained("orai-nlp/Llama-eus-8B", padding_side="left")
model_inputs = tokenizer(["Kolore zerrenda bat: gorria, urdina"], return_tensors="pt").to(DEVICE)

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Kolore zerrenda bat: gorria, urdina, berdea, horia, beltza, zuria, arrosa, morea,'

## Embeddings

In [2]:
import transformers
import torch

# There are important!
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [3]:
from transformers import AutoTokenizer, AutoModel
from accelerate.test_utils.testing import get_backend

DEVICE, _, _ = get_backend() # automatically detects the underlying device type (CUDA, CPU, XPU, MPS, etc.)
print(DEVICE)

tokenizer = AutoTokenizer.from_pretrained("orai-nlp/Llama-eus-8B")
model = AutoModel.from_pretrained("orai-nlp/Llama-eus-8B", torch_dtype=torch.bfloat16, device_map="auto")

cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# input text
esaldi = "Pilotan Altuna III. atera da garaile, Iñaki Artola 22-19 garaitu ondoren. Bi gipuzkoarrek urte luzeetan ahaztuko ez dugun erakustaldia eman ziguten."


inputs = tokenizer(esaldi, return_tensors="pt")

  # text embedding
with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state

X = embeddings.sum(axis=1).cpu().float().numpy()
print(X.shape)
with open('/content/drive/MyDrive/Colab Notebooks/altuna.bin', 'wb') as f:
    np.save(f,X)

(1, 4096)


In [ ]:
# input text
esaldi = "Pilotan Altuna atera da garaile."

tokens = tokenizer.tokenize(esaldi)
print("Tokens:", tokens)

Tokens: ['P', 'ilot', 'an', 'ĠAlt', 'una', 'Ġa', 'tera', 'Ġda', 'Ġg', 'ara', 'ile', '.']


In [ ]:
# input text
esaldi = "Hegoafrikan ardo ona egiten omen da Stellenbosch aldean. Lurmutur hiria, Pretoria, hiri handiak ere topa daitezke"


inputs = tokenizer(esaldi, return_tensors="pt")

  # text embedding
with torch.no_grad():
    embeddings = model(**inputs).last_hidden_state

X = embeddings.sum(axis=1).cpu().float().numpy()
print(X.shape)
with open('/content/drive/MyDrive/Colab Notebooks/hegoafrika.bin', 'wb') as f:
    np.save(f,X)

(1, 4096)


In [ ]:
X = embeddings.cpu().float().numpy()
print(X.shape, inputs.data['input_ids'].shape)
print(X.sum(axis=1))

(1, 22, 4096) torch.Size([1, 22])
[[-23.628555   -8.2438965  -4.425537  ...  15.032227   14.272949
   -2.8916016]]


In [ ]:
embeddings.sum(axis=1).cpu().float().numpy().shape

(1, 4096)

In [ ]:
import os
print(os.getcwd())
for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        print(os.path.join(root, file))

/content
/content/.config/.last_opt_in_prompt.yaml
/content/.config/gce
/content/.config/.last_update_check.json
/content/.config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
/content/.config/config_sentinel
/content/.config/default_configs.db
/content/.config/active_config
/content/.config/.last_survey_prompt.yaml
/content/.config/configurations/config_default
/content/.config/logs/2025.05.29/14.01.02.146440.log
/content/.config/logs/2025.05.29/14.01.01.430943.log
/content/.config/logs/2025.05.29/14.00.52.256341.log
/content/.config/logs/2025.05.29/14.00.20.854017.log
/content/.config/logs/2025.05.29/14.00.51.008524.log
/content/.config/logs/2025.05.29/14.00.41.804176.log
/content/drive/MyDrive/Apunteak.pdf
/content/drive/MyDrive/Dokumentu izengabea.gdoc
/content/drive/MyDrive/URTZI_FlashPres.pptx
/content/drive/MyDrive/Urtzi_Jauregi_PLANTILLA ANEXOS MÁSTER UB-CSIC Urtzi.pdf
/content/drive/MyDrive/delft.gdoc
/content/drive/MyDrive/Delft article.gdoc
/content/drive/M

In [ ]:
with open ("/content/drive/MyDrive/Colab Notebooks/gizartea_2.jsonl", "r") as file: #wikipedia
    gizartea = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/ekonomia_2.jsonl", "r") as file: #wikipedia
    ekonomia = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/geografia_2.jsonl", "r") as file: #wikipedia
    geografia = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/politika_2.jsonl", "r") as file: #wikipedia
    politika = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/kirola_2.jsonl", "r") as file: #wikipedia
    kirola = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/other_2.jsonl", "r") as file: #wikipedia
    other = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/zientzia_teknologia_2.jsonl", "r") as file: #wikipedia
    zientzia_teknologia = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/historia_2.jsonl", "r") as file: #wikipedia
    historia = json.load(file)

### Geografia (48 / 66 esaldi)

In [ ]:
# input text
embed_geografia = np.empty((0,4096))
excep_geografia = []
i= 0
for esaldi in geografia:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_geografia = np.vstack((embed_geografia,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_geografia.append(esaldi)
        torch.cuda.empty_cache()
        continue

  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_geografia.shape)
with open('/content/drive/MyDrive/Colab Notebooks/geografia.bin', 'wb') as f:
    np.save(f,embed_geografia)


(66, 4096)


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/geografia.bin', 'rb') as f:
    X = np.load(f)


### Historia (118 / 102 esaldi)

In [ ]:
# input text
embed_historia = np.empty((0,4096))
excep_historia = []
i = 0
for esaldi in historia:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_historia = np.vstack((embed_historia,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_historia.append(esaldi)
        torch.cuda.empty_cache()
        continue

  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_historia.shape)
with open('/content/drive/MyDrive/Colab Notebooks/historia.bin', 'wb') as f:
    np.save(f,embed_historia)


(102, 4096)


### Zientzia eta teknologia (151 / 150 esaldi)

In [ ]:
# input text
embed_zientzia = np.empty((0,4096))
excep_zientzia_teknologia = []
i = 0
for esaldi in zientzia_teknologia:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_zientzia = np.vstack((embed_zientzia,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_zientzia_teknologia.append(esaldi)
        torch.cuda.empty_cache()
        continue

  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_zientzia.shape)
with open('/content/drive/MyDrive/Colab Notebooks/zientzia_teknologia.bin', 'wb') as f:
    np.save(f,embed_zientzia)


(150, 4096)


### Ekonomia (239 / 238 esaldi)

In [ ]:
# input text
embed_ekonomia = np.empty((0,4096))
excep_ekonomia = []
i = 0
for esaldi in ekonomia:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_ekonomia = np.vstack((embed_ekonomia,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_ekonomia.append(esaldi)
        torch.cuda.empty_cache()
        continue

  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_ekonomia.shape)
with open('/content/drive/MyDrive/Colab Notebooks/ekonomia.bin', 'wb') as f:
    np.save(f,embed_ekonomia)


(238, 4096)


In [ ]:
print(len(kirola))
print(len(kirola[:290]))
print(len(kirola[290:]))

592
290
302


In [ ]:
print(len(kirola[266].split()))

857


### Kirola (592 / 494 esaldi)



In [ ]:
# input text
embed_kirola = np.empty((0,4096))
excep_kirol = []
i = 0
for esaldi in kirola:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_kirola = np.vstack((embed_kirola,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_kirol.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_kirola.shape)
with open('/content/drive/MyDrive/Colab Notebooks/kirola.bin', 'wb') as f:
    np.save(f,embed_kirola)


(494, 4096)


In [ ]:
with open("/content/drive/MyDrive/Colab Notebooks/salbuespen_kirola.jsonl", "w") as f:
  json.dump(excep_kirol,f)

### Politika (732 / 682  esaldi)

In [ ]:
# input text
embed_politika = np.empty((0,4096))
excep_politika = []
i = 0
for esaldi in politika:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_politika = np.vstack((embed_politika,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_politika.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_politika.shape)
with open('/content/drive/MyDrive/Colab Notebooks/politika.bin', 'wb') as f:
    np.save(f,embed_politika)
with open("/content/drive/MyDrive/Colab Notebooks/salbuespen_politika.jsonl", "w") as f:
  json.dump(excep_politika,f)

(682, 4096)


### Gizartea (1496 / 1528 esaldi)

In [ ]:
# input text
embed_gizartea = np.empty((0,4096))
excep_gizartea = []
i = 0
for esaldi in gizartea:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_gizartea = np.vstack((embed_gizartea,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_gizartea.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_gizartea.shape)
with open('/content/drive/MyDrive/Colab Notebooks/gizartea.bin', 'wb') as f:
    np.save(f,embed_gizartea)
with open("/content/drive/MyDrive/Colab Notebooks/salbuespen_gizartea.jsonl", "w") as f:
  json.dump(excep_gizartea,f)

Out of memory 550. 
Out of memory 1314. 
(1526, 4096)


### Sentiment Analysis

In [4]:
with open ("/content/drive/MyDrive/Colab Notebooks/sentiment/positive.jsonl", "r") as file: #positibo
    positive = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/sentiment/negative.jsonl", "r") as file: #negatibo
    negative = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/sentiment/neutral.jsonl", "r") as file: #neutroa
    neutral = json.load(file)


### **Positibo** 130 esaldi

In [5]:
# input text
embed_positibo = np.empty((0,4096))
excep_positibo = []
i = 0
for esaldi in positive:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_positibo = np.vstack((embed_positibo,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_positibo.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_positibo.shape)
with open('/content/drive/MyDrive/Colab Notebooks/sentiment/positive.bin', 'wb') as f:
    np.save(f,embed_positibo)
with open("/content/drive/MyDrive/Colab Notebooks/sentiment/salbuespen_positive.jsonl", "w") as f:
  json.dump(excep_positibo,f)

(130, 4096)


### **Negatibo** 103 esaldi

In [6]:
# input text
embed_negatibo = np.empty((0,4096))
excep_negatibo = []
i = 0
for esaldi in negative:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_negatibo = np.vstack((embed_negatibo,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_positibo.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_negatibo.shape)
with open('/content/drive/MyDrive/Colab Notebooks/sentiment/negative.bin', 'wb') as f:
    np.save(f,embed_negatibo)
with open("/content/drive/MyDrive/Colab Notebooks/sentiment/salbuespen_negative.jsonl", "w") as f:
  json.dump(excep_negatibo,f)

(103, 4096)


### **Neutro** 103 esaldi

In [7]:
# input text
embed_neutro = np.empty((0,4096))
excep_neutro = []
i = 0
for esaldi in neutral:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_neutro = np.vstack((embed_neutro,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_neutro.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_neutro.shape)
with open('/content/drive/MyDrive/Colab Notebooks/sentiment/neutral.bin', 'wb') as f:
    np.save(f,embed_neutro)
with open("/content/drive/MyDrive/Colab Notebooks/sentiment/salbuespen_neutral.jsonl", "w") as f:
  json.dump(excep_neutro,f)

(103, 4096)


## **ChatGPT**-rekin eraikitako esaldiak

In [8]:
with open ("/content/drive/MyDrive/Colab Notebooks/sentiment/positive_gpt.jsonl", "r") as file: #positibo
    positive_gpt = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/sentiment/negative_gpt.jsonl", "r") as file: #negatibo
    negative_gpt = json.load(file)
with open ("/content/drive/MyDrive/Colab Notebooks/sentiment/neutral_gpt.jsonl", "r") as file: #neutroa
    neutral_gpt = json.load(file)


### **Positibo** 178 esaldi

In [9]:
# input text
embed_positibo = np.empty((0,4096))
excep_positibo = []
i = 0
for esaldi in positive_gpt:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_positibo = np.vstack((embed_positibo,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_positibo.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_positibo.shape)
with open('/content/drive/MyDrive/Colab Notebooks/sentiment/positive_gpt.bin', 'wb') as f:
    np.save(f,embed_positibo)
with open("/content/drive/MyDrive/Colab Notebooks/sentiment/salbuespen_positive_gpt.jsonl", "w") as f:
  json.dump(excep_positibo,f)

(178, 4096)


### **Negatibo** 180 esaldi

In [10]:
# input text
embed_positibo = np.empty((0,4096))
excep_positibo = []
i = 0
for esaldi in negative_gpt:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_positibo = np.vstack((embed_positibo,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_positibo.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_positibo.shape)
with open('/content/drive/MyDrive/Colab Notebooks/sentiment/negative_gpt.bin', 'wb') as f:
    np.save(f,embed_positibo)
with open("/content/drive/MyDrive/Colab Notebooks/sentiment/salbuespen_negative_gpt.jsonl", "w") as f:
  json.dump(excep_positibo,f)

(180, 4096)


### **Neutro** 180 esaldi

In [11]:
# input text
embed_positibo = np.empty((0,4096))
excep_positibo = []
i = 0
for esaldi in neutral_gpt:
  i +=1
  try:
    inputs = tokenizer(esaldi, return_tensors="pt")

    # text embedding
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state

    X = embeddings.sum(axis=1).cpu().float().numpy()
    embed_positibo = np.vstack((embed_positibo,X))
  except torch.cuda.OutOfMemoryError:
        print(f"Out of memory {i-1}. ")
        excep_positibo.append(esaldi)
        torch.cuda.empty_cache()
        continue
  del inputs,embeddings, X
  torch.cuda.empty_cache()
  gc.collect()
print(embed_positibo.shape)
with open('/content/drive/MyDrive/Colab Notebooks/sentiment/neutral_gpt.bin', 'wb') as f:
    np.save(f,embed_positibo)
with open("/content/drive/MyDrive/Colab Notebooks/sentiment/salbuespen_neutral_gpt.jsonl", "w") as f:
  json.dump(excep_positibo,f)

(180, 4096)
